In [ ]:
### Part 1: embedding a trained generative AI model into a music app

In [1]:
from app.audio import create_audio_engine

In [3]:
import urllib.request

In [4]:
urllib.request.urlretrieve("https://play.forum.ircam.fr/rave-vst-api/get_model/darbouka_onnx", "darbouka_onnx.ts")

('darbouka_onnx.ts', <http.client.HTTPMessage at 0x12afb6930>)

In [2]:
audio_engine = create_audio_engine("test-loop.wav", "darbouka_onnx.ts")

In [3]:
audio_engine.start()

In [4]:
audio_engine.loop.set()

In [5]:
audio_engine.transform.set()

In [6]:
audio_engine.set_latent_coordinates([0.1, 0.1, 0.5, 0.5])

In [7]:
audio_engine.transform.clear()

In [8]:
audio_engine.stop.set()

In [19]:
audio_engine.stop.clear()

In [ ]:
### Part 2: Training a model 

In [8]:
import copy
import numpy as np
import torch
import torch.nn as nn
import json

In [6]:
import app.control.xy

In [9]:
class Model(nn.Module):
    def __init__(self, *a, **kw):
        super().__init__(*a, **kw)

        self.layers = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, 4),
        )

    def forward(self, x):
        """
        Forward pass
        """
        return self.layers(x)